## PROMPT
Generate a pipeline for Databricks:
* Get Data information from two web addresses:
   * Space launches: https://api.spacexdata.com/v3/launches
* Filter the list of launches based on launch year and launch success status.
* Send the chosen information to the web address: https://httpbin.org/post
* The script must provide status updates on its progress, report any errors encountered, confirm the outcome of the final data sending step, and measure/report execution times.

In [0]:
import requests
import time
from datetime import datetime

# --- Configuration ---
SPACEX_API_URL = "https://api.spacexdata.com/v3/launches"
POST_URL = "https://httpbin.org/post"
FILTER_YEAR = "2020"
FILTER_SUCCESS = True

# --- Helper Functions ---
def log(msg):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

def fetch_launch_data():
    log("Fetching SpaceX launch data...")
    try:
        response = requests.get(SPACEX_API_URL)
        response.raise_for_status()
        log("Successfully fetched launch data.")
        return response.json()
    except requests.RequestException as e:
        log(f"Error fetching launch data: {e}")
        return []

def filter_launches(data, year, success_status):
    log(f"Filtering data for year={year} and success={success_status}...")
    filtered = [
        launch for launch in data
        if str(launch.get("launch_year")) == year and launch.get("launch_success") == success_status
    ]
    log(f"Filtered down to {len(filtered)} launches.")
    return filtered

def send_filtered_data(filtered_data):
    log("Sending filtered data to POST endpoint...")
    try:
        response = requests.post(POST_URL, json=filtered_data)
        response.raise_for_status()
        log("Data successfully sent.")
        return response.json()
    except requests.RequestException as e:
        log(f"Error sending data: {e}")
        return None

# --- Main Pipeline ---
def main():
    total_start = time.time()

    fetch_start = time.time()
    data = fetch_launch_data()
    fetch_time = time.time() - fetch_start

    if not data:
        log("Terminating due to fetch failure.")
        return

    filter_start = time.time()
    filtered_data = filter_launches(data, FILTER_YEAR, FILTER_SUCCESS)
    filter_time = time.time() - filter_start

    send_start = time.time()
    response = send_filtered_data(filtered_data)
    send_time = time.time() - send_start

    total_time = time.time() - total_start

    log(f"Fetch time: {fetch_time:.2f}s")
    log(f"Filter time: {filter_time:.2f}s")
    log(f"Send time: {send_time:.2f}s")
    log(f"Total execution time: {total_time:.2f}s")

    if response:
        log("Final Response Preview:")
        print(response)

# Run the pipeline
main()